In [1]:
import os
os.environ['KERAS_BACKEND'] = 'tensorflow'

In [2]:
import numpy as np
import pandas as pd
import keras_core as keras
import tensorflow as tf
import keras_nlp
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

Using TensorFlow backend


In [3]:
df_train = pd.read_csv("/kaggle/input/train-and-test-data/train.csv")
df_test = pd.read_csv("/kaggle/input/train-and-test-data/test.csv")

In [11]:
df_train.sample(n=30)

,id,keyword,location,text,target
5027,7168,mudslide,London,#GBBO The difference between Paul and Mary my ...,0
6635,9502,terrorist,NaN,Fresh encounter in Pulwama of J&amp;amp;K one ...,1
6282,8973,storm,#BossNation!,Finna storm. Fuck my back boutta start hurting...,0
7140,10229,volcano,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
545,794,avalanche,NaN,Chevrolet : Avalanche LT 2011 lt used 5.3 l v ...,1
4400,6256,hijacking,Brazil,#hot Funtenna: hijacking computers to send da...,0
5608,8002,razed,NaN,The Latest: More homes razed by Northern Calif...,1
4538,6452,injured,NaN,@chikislizeth08 you're not injured anymore? ??,0
4231,6009,hazardous,"Muntinlupa City, Philippines",@HearItFromPatty Also pls help us w/ our campa...,0
715,1035,bleeding,AZ,Joe Landolina: This gel can make you stop blee...,0


In [6]:
df_test.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [36]:
# Create a new 'length' column into the train and test data
df_train['length'] = df_train["text"].apply(lambda x : len(x))
df_test['length'] = df_train["text"].apply(lambda x : len(x))



In [37]:
BATCH_SIZE = 32
num_training_examples = df_train.shape[0] # 7613 training examples 
train_split = 0.8
test_split = 0.2
steps_per_epoch = int(num_training_examples)* train_split // batch_size

EPOCHS = 1
AUTO = tf.data.experimental.AUTOTUNE

In [38]:
from sklearn.model_selection import train_test_split

X = df_train["text"]
y = df_train["target"]

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=test_split, random_state=42)

X_test = df_test["text"]

In [47]:
preset = "distil_bert_base_en_uncased"

preprocessor = keras_nlp.models.DistilBertPreprocessor.from_preset(preset, sequence_length = 160, name="tweet_preprocessor")
classifier = keras_nlp.models.DistilBertClassifier.from_preset(preset, preprocessor = preprocessor, num_classes = 2)


In [50]:
from keras.optimizers import Adam

classifier.compile(loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
                  optimizer=Adam(learning_rate=1e-5),
                   metrics=['accuracy'])

model_fit = classifier.fit(x = X_train, y = y_train, batch_size = BATCH_SIZE, epochs=EPOCHS, validation_data=(X_val, y_val))


191/191 ━━━━━━━━━━━━━━━━━━━━ 3915s 20s/step - accuracy: 0.7762 - loss: 0.5421 - val_accuracy: 0.8444 - val_loss: 0.3941


In [58]:
import joblib

# Save the model
joblib.dump(model_fit, '/kaggle/working/model.pkl')



['/kaggle/working/model.pkl']

In [54]:
y_pred_train = classifier.predict(X_train)

KeyboardInterrupt: 